In [1]:
#import libraries
import os, io, re, tqdm, datetime, nltk, torch, transformers #nltk mecbur olmasaz yuklemeyin

import pandas as pd, numpy as np, matplotlib as mpl, matplotlib.pyplot as plt, seaborn as sns

from torch import nn
from torchtext import vocab
from transformers import BertTokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data import get_tokenizer
from torch.utils.data import DataLoader, Dataset, random_split
from torchtext.data.functional import to_map_style_dataset
from torch.optim import Adam, Adagrad, RMSprop, SGD

#ignore warnings
import warnings
warnings.filterwarnings(action= 'ignore')

#set parameters for "Matplotlib"
mpl.rcParams['figure.figsize']  = [16, 9]

In [2]:
# pip install torch torchtext -f https://download.pytorch.org/whl/torch_stable.html
# pip uninstall torch torchtext -y
# pip install torch==2.1.0 torchtext==0.16.0

### About Dataset
#### AG's News Topic Classification Dataset
### ORIGIN
AG is a collection of more than 1 million news articles. News articles have been gathered from more than 2000 news sources by ComeToMyHead in more than 1 year of activity. ComeToMyHead is an academic news search engine which has been running since July, 2004. The dataset is provided by the academic comunity for research purposes in data mining (clustering, classification, etc), information retrieval (ranking, search, etc), xml, data compression, data streaming, and any other non-commercial activity. For more information, please refer to the link http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html .

The AG's news topic classification dataset is constructed by Xiang Zhang (xiang.zhang@nyu.edu) from the dataset above. It is used as a text classification benchmark in the following paper: Xiang Zhang, Junbo Zhao, Yann LeCun. Character-level Convolutional Networks for Text Classification. Advances in Neural Information Processing Systems 28 (NIPS 2015).

### DESCRIPTION
The AG's news topic classification dataset is constructed by choosing 4 largest classes from the original corpus. Each class contains 30,000 training samples and 1,900 testing samples. The total number of training samples is 120,000 and testing 7,600.

The file classes.txt contains a list of classes corresponding to each label.

The files train.csv and test.csv contain all the training samples as comma-sparated values. There are 3 columns in them, corresponding to class index (1 to 4), title and description. The title and description are escaped using double quotes ("), and any internal double quote is escaped by 2 double quotes (""). New lines are escaped by a backslash followed with an "n" character, that is "\n".

#### About this file

This file consists of 7600 testing samples of news articles that contain 3 columns. The first column is Class Id, the second column is Title and the third column is Description. 

**The class ids are numbered 1-4 where**
* 1 ------------> World
* 2 ------------> Sports
* 3 ------------> Business
* 4 ------------> Sci/Tech.

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
df = pd.concat([train_df, test_df], axis= 0)
df.index = range(df.shape[0])
df['text'] = 'The title of news is ' + df['Title'] + ' .The news are below ' +  df['Description']
df.rename({'Class Index': 'label'}, axis= 1, inplace= True)
df.drop(['Title', 'Description'], axis= 1, inplace= True)
df = df[['text', 'label']]
df['label'] = df['label'] - 1
# df.to_csv('Final_df.csv', index= False)

In [4]:
df['text'].str.len().min()

58

In [5]:
df.loc[idx, 'text']

NameError: name 'idx' is not defined

In [ ]:
df.loc[idx, 'text']

In [ ]:
BERT_PATH = 'bert-base-uncased'
TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)
vectorized_text = TOKENIZER.encode_plus(df.loc[idx, 'text'], None,
                   add_special_tokens=True,
                   max_length = 192,
                   pad_to_max_length=True,
                   truncation='longest_first',), 

In [ ]:
# [101, 1996, 2516, 1997, 2739, 2003, 2678, 2399, 1005, 2204, 2005, 2336, 1005, 1012, 1996, 2739, 2024, 2917, 3274, 2399, 2064, 5326, 3291, 1011, 13729, 1998, 2136, 1011, 2311, 1999, 2336, 1010, 2360, 2399, 3068, 8519, 1012]

In [ ]:
vectorized_text[0].keys()

In [ ]:
vectorized_text[0]

In [ ]:
np.array(vectorized_text[0]['input_ids']).shape

In [14]:
BERT_PATH = 'bert-base-uncased'
TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)
#reqemlere cevirsin deye

class GetDataset():
    def __init__(self, dataset, tokenizer, MAX_LEN):
        self.label = dataset['label']
        self.text = dataset['text']
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
        
        
    def __len__(self):
        return len(self.label) #tells pytorch how many rows does dataframe have
    
    def __getitem__(self, idx): #iterable-style dataset and map-style - __iter__ & __len__ 
        label = self.label.iloc[idx]
        text = self.text.iloc[idx]
        vectorized_text = TOKENIZER.encode_plus(text, None,
                                               add_special_tokens= True,
                                               max_length = self.max_len,
                                               pad_to_max_length=True,
                                               truncation='longest_first',), 
        input_ids = vectorized_text[0]['input_ids']
        attention_masks = vectorized_text[0]['attention_mask']
#         tokenized_text = tokenizer(text)
        
        
        return {'ids': torch.tensor(input_ids),
                'attention_masks': attention_masks,
                'labels': torch.tensor(label)}
    
dataset = GetDataset(df, tokenizer= TOKENIZER, MAX_LEN= 192)
dataset

## 📌 Key Parameters

### 🔹 Short Explanation

- **`vocab_size`**: Number of unique words (tokens) in your dataset's vocabulary.
- **`embed_dim`**: Size of each word embedding vector (e.g., 100, 300).
- **`num_class`**: Number of output categories (labels) for classification.

---

### 🔍 Detailed Explanation

#### 1. 🧾 `vocab_size` — Vocabulary Size
- Refers to the **total number of unique tokens** (words, subwords, or characters) in your dataset.
- After tokenization, you build a vocabulary.  
- Example: If you have 10,000 unique words, then `vocab_size = 10000`.
- In `nn.Embedding`, this tells PyTorch **how many rows** to create in the embedding matrix (one for each token).

---

#### 2. 📏 `embed_dim` — Embedding Dimension
- Each token is converted into a **dense vector** of fixed size.
- `embed_dim` defines **how many features** each word is represented by.
- Common values: `50`, `100`, `300`, `768` (used in BERT).
- If `embed_dim = 100`, then:
  - "king" → `[0.12, -0.45, ..., 0.33]` (a 100-dimensional vector)

✅ Embeddings help capture **semantic meaning** — similar words have similar vectors.

---

#### 3. 🎯 `num_class` — Number of Output Classes
- Defines the number of **categories/labels** your model predicts.
  - For binary classification → `num_class = 2`
  - For sentiment classification (pos/neg/neutral) → `num_class = 3`
  - For topic classification → `num_class = N`

- The final layer is often:
  ```python
  nn.Linear(in_features, num_class)


## Embedding Dimensonlarda reqem nece yaranir? (Embedding layer)

## Define the model

In [7]:
file_path = 'Final_df.csv'
tokenizer = get_tokenizer('basic_english') #sozlere ayirmaq ucun

def yield_tokens(file_path):
    with io.open(file_path) as f:
        for line in f:
            yield tokenizer(line.strip())
            
vocab = build_vocab_from_iterator(yield_tokens(file_path), specials= ['<unk>'])
vocab_size = len(vocab)

In [42]:
my_transformer_layer = nn.TransformerEncoderLayer(
    d_model=768,  # embedding size
    nhead=12      # number of attention heads
)

class NewsClassificationGRU(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, num_heads):
        super(NewsClassificationGRU, self).__init__()
        
        self.embedding_layer = nn.Embedding(num_embeddings= vocab_size, embedding_dim= embed_dim)
        
        self.transformer_layer = nn.TransformerEncoderLayer(
                            d_model= embed_dim, 
                            nhead= num_heads
        )
        
        self.gru_layer = nn.GRU(embed_dim, hidden_size= 128, num_layers= 2, batch_first= True)  #RNN, GRU, LSTM - sequence tipli datalarla ishlemek ucun
                                                                             #istifade olunur. Sequence - NLP ve Time Series
        self.dense_layer = nn.Linear(128, num_class)
        self.softmax_act_fc = nn.Softmax(dim= 1)
        
    def forward(self, input_ids, attention_mask):
        X = self.embedding_layer(input_ids)
        X = X.transpose(0, 1)   # [seq_len, batch, embed_dim]
        mask = (attention_mask == 0)   # shape should be [batch, seq_len]
        if mask.shape[0] != X.shape[1]:   # X.shape[1] = batch
            mask = mask.transpose(0, 1)   # fix if it’s swapped accidentally

        X = self.transformer_layer(X, src_key_padding_mask=mask)
#         X = self.transformer_layer(
#             X,
#             src_key_padding_mask=(attention_mask == 0)   # [batch, seq_len] ✅
#         )

        X = X.transpose(0, 1)   
        
#   
        X, _ = self.gru_layer(X)              # batch_first=True → [batch, seq_len, hidden_size]
        X = X[:, -1, :]                    # last time step [batch, hidden_size]
        X = self.dense_layer(X) 
# #         X, _ = self.gru_layer(X) #return tuple - 2 dene shey qaytarir
# #         X = self.dense_layer(X)
# #         X = X[:, -1 ,:]
#         X = self.softmax_act_fc(X)
        return X
    
model = NewsClassificationGRU(vocab_size= vocab_size, 
                              embed_dim= 64, 
                              num_class= 4, 
                              num_heads= 32)
model

NewsClassificationGRU(
  (embedding_layer): Embedding(98628, 64)
  (transformer_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
    )
    (linear1): Linear(in_features=64, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=64, bias=True)
    (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (gru_layer): GRU(64, 128, num_layers=2, batch_first=True)
  (dense_layer): Linear(in_features=128, out_features=4, bias=True)
  (softmax_act_fc): Softmax(dim=1)
)

In [30]:
model.forward(ids, attention_mask).shape

torch.Size([128, 4])

In [ ]:
attention_mask.uns

In [ ]:
torch.tensor(attention_mask)

In [21]:
model.forward()

TypeError: NewsClassificationGRU.forward() missing 2 required positional arguments: 'input_ids' and 'attention_mask'

In [45]:
BATCH_SIZE = 128
LEARNING_RATE = 0.01
EPOCHS = 3
NUM_WORKERS = 0
optimizer = SGD(model.parameters(), lr= LEARNING_RATE)
loss_fn = torch.nn.CrossEntropyLoss()

train_size = int(df.shape[0] * 0.9)
train_subset, test_subset = random_split(dataset, lengths= [train_size, df.shape[0] - train_size])

train_data_loader = DataLoader(train_subset, batch_size= BATCH_SIZE, num_workers = NUM_WORKERS )
test_data_loader = DataLoader(test_subset, batch_size= BATCH_SIZE, num_workers= NUM_WORKERS)

## Train the model

In [ ]:
next(iter(train_data_loader))

In [ ]:
# predictions

In [ ]:
predictions.shape

In [ ]:
y.shape

In [ ]:
train_data_loader

In [ ]:
predictions.shape

In [ ]:
labels.shape

In [ ]:
# predictions

In [46]:
model.train()

for epoch_n in range(1, EPOCHS + 1):
    print(f'Epoch {epoch_n} ----------------------------->\n')
    for idx, (batch) in enumerate(train_data_loader):
        ids = batch['ids']
        attention_mask = batch['attention_masks']
        attention_mask = torch.stack(attention_mask, dim=0)
        
        labels = batch['labels']
        
        predictions = model(ids, attention_mask)
        loss = loss_fn(predictions, labels)

        #backward propogation
        loss.backward()
        optimizer.zero_grad()
        
        
        if idx % 100 == 0 and idx != 0:
            print(f"Loss for batch {idx}: {round(loss.item(), 2)}")

Epoch 1 ----------------------------->

Loss for batch 100: 1.39
Loss for batch 200: 1.38
Loss for batch 300: 1.38
Loss for batch 400: 1.38
Loss for batch 500: 1.39
Loss for batch 600: 1.38
Loss for batch 700: 1.38
Loss for batch 800: 1.39
Epoch 2 ----------------------------->

Loss for batch 100: 1.38
Loss for batch 200: 1.38
Loss for batch 300: 1.38
Loss for batch 400: 1.38
Loss for batch 500: 1.38
Loss for batch 600: 1.38
Loss for batch 700: 1.38
Loss for batch 800: 1.38
Epoch 3 ----------------------------->

Loss for batch 100: 1.36
Loss for batch 200: 1.33
Loss for batch 300: 1.32
Loss for batch 400: 1.34
Loss for batch 500: 1.26
Loss for batch 600: 1.24
Loss for batch 700: 1.2
Loss for batch 800: 1.16


## Testing

In [49]:
accuracy_value = 0
test_size = df.shape[0] - train_size

model.eval()
with torch.no_grad():
    for idx, batch in enumerate(test_data_loader):
        ids = batch['ids']
        attention_mask = batch['attention_masks']
        attention_mask = torch.stack(attention_mask, dim=0)
        
        labels = batch['labels']
        
        predictions = model(ids, attention_mask)
        accuracy_value += (predictions.argmax(1) == labels).sum().item()
        if idx % 25 == 0 and idx != 0:  
            print(f"Accuracy Value: {accuracy_value} out of {idx  * BATCH_SIZE}") 

    print(f"\nAccuracy Score: {round((accuracy_value / test_size) * 100, 2)}")

Accuracy Value: 1388 out of 3200
Accuracy Value: 2705 out of 6400
Accuracy Value: 4070 out of 9600

Accuracy Score: 41.71
